In [44]:
import lmql
import asyncio
import os
from dotenv import load_dotenv
load_dotenv()

True

# Simple token constraint

In [7]:
@lmql.query
async def tipo_test():
    '''lmql
    argmax
        "Pregunta: Sevilla es una ciudad ubicada en que parte de Espana \n"
        "A) Sur \n"
        "B) Norte \n"
        "C) Este \n"
        "D) Afuera de Espana \n"
        "Respuesta: [RESPUESTA]"
    from
        "openai/text-davinci-003"
    where
        RESPUESTA in set(["A", "B", "C", "D"])
    '''

In [8]:
r = await tipo_test()

In [9]:
r

[LMQLResult(prompt='Pregunta: Sevilla es una ciudad ubicada en que parte de Espana \nA) Sur \nB) Norte \nC) Este \nD) Afuera de Espana \nRespuesta: A', variables={'RESPUESTA': 'A'}, distribution_variable=None, distribution_values=None)]

In [10]:
r[0].variables

{'RESPUESTA': 'A'}

# Distribution

In [20]:
@lmql.query
async def sentiment_analysis(input):
    '''lmql
    argmax
        "Classifica el sentimiento de la siguiente frase como POS, NEG O NEUT: \n"
        "{input} \n"
        "El sentimento de la frase es [SENTIMENT]"
    from
        "openai/text-davinci-003"
    distribution
        SENTIMENT in ["POS", "NEUT", "NEG"]
    '''

In [21]:
r = await sentiment_analysis('Estoy muy feliz hoy!')

In [22]:
r.variables.keys()

dict_keys(['SENTIMENT', 'P(SENTIMENT)', 'log P(SENTIMENT)'])

In [24]:
r.variables.get('P(SENTIMENT)')

[('POS', 0.3336283714733308),
 ('NEUT', 0.332771130433606),
 ('NEG', 0.3336004980930632)]

# JSON response

In [25]:
from dataclasses import dataclass

In [32]:
# Only works in playground :p ?? 
@lmql.query
async def jsonize():
    '''lmql
    from dataclasses import dataclass
    import lmql

    @dataclass
    class Event:
        nombre: str
        descripcion: str
            
    @dataclass
    class Meetup:
        nombre: str
        organizadores: str
        event: Event
            
    argmax
        "Por favor, coloca el texto a continuación en un formato JSON estructurado: \n"
        "IA Generativa Sevilla es un grupo de meetup colocada en Sevilla, Espana. Los organizadores son Ivan y Mathias \n"
        "El evento de hoy trata de dominando un LLM utilizando LMQL \n"
        "structured: [MEETUP]"
    from
        "openai/text-davinci-003"
    where
        type(MEETUP) is Meetup
    '''

AssertionError: operator (type(MEETUP) is Meetup)
 is not supported.

In [33]:
@lmql.query
async def jsonize():
    '''lmql
    argmax 
        """
        Por favor, coloca el texto a continuación en un formato JSON estructurado:

        IA Generativa Sevilla es un grupo de meetup colocada en Sevilla, Espana. Los organizadores son Ivan y Mathias
        El evento de hoy trata de dominando un LLM utilizando LMQL

        {{
        "meetup-name": "[NAME]",
        "location": "[LOCATION]",
        "organizers": "[ORGANIZERS]",
        "description": "[DESCRIPTION]",
        }}
        """
    from
        "openai/text-davinci-003" 
    where
        STOPS_BEFORE(NAME, '"') and STOPS_BEFORE(DESCRIPTION, '"') and STOPS_BEFORE(LOCATION, '"') and STOPS_BEFORE(ORGANIZERS, '"') and len(NAME) < 25
    '''

In [34]:
r = await jsonize()
r

[LMQLResult(prompt='\nPor favor, coloca el texto a continuación en un formato JSON estructurado:\nIA Generativa Sevilla es un grupo de meetup colocada en Sevilla, Espana. Los organizadores son Ivan y Mathias\nEl evento de hoy trata de dominando un LLM utilizando LMQL\n{\n"meetup-name": "IA Genera",\n"location": "Sevilla, Espana",\n"organizers": "Ivan y Mathias",\n"description": "Dominando un LLM utilizando LMQL",\n}\n', variables={'NAME': 'IA Genera', 'LOCATION': 'Sevilla, Espana', 'ORGANIZERS': 'Ivan y Mathias', 'DESCRIPTION': 'Dominando un LLM utilizando LMQL'}, distribution_variable=None, distribution_values=None)]

In [38]:
r[0].variables

{'NAME': 'IA Genera',
 'LOCATION': 'Sevilla, Espana',
 'ORGANIZERS': 'Ivan y Mathias',
 'DESCRIPTION': 'Dominando un LLM utilizando LMQL'}

# Calling function / control flow

In [39]:
import requests

In [40]:
url = "https://api.dictionaryapi.dev/api/v2/entries/en/{word}"

In [86]:
async def get_definitions(word):
    data = requests.get(url.format(word='asynchronous')).json()
    # Extract the definitions
    definitions = []
    for meaning in data[0]['meanings'][0]['definitions']:
        definitions.append(f"definition: {meaning['definition']}")
    return definitions

In [108]:
@lmql.query
async def example_sentence(term):
    '''lmql
    argmax
        """
        Please provide an example sentences for each definition of the word: {term}
        """

        definitions = await get_definitions(term)
        definition_strings = "\n".join(definitions)

        """
        Here are the dictionary definitions:
        {definition_strings}
        """
        
        examples = []
        for i in range(len(definitions)):
            "-[EXAMPLE]" where STOPS_AT(EXAMPLE, "\n")
            examples.append(EXAMPLE.strip())
    from
        "openai/text-davinci-003"
    '''

In [115]:
response = await example_sentence("Asynchronous")